# LangChain Agents

In this notebook, we'll build a basic LangChain agent. We'll leverage OpenAI for LLM responses and implement various tools to create a flexible, dynamic workflow. LangChain's agent framework will allow us to orchestrate different functions and API calls seamlessly.

In [1]:
from langchain_community.chat_models import ChatOpenAI
import os
from rag import RagEngine 
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv
load_dotenv()

True

### Initializing the LLM

In [2]:
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0,
    model_name="gpt-3.5-turbo"
)

/var/folders/61/42wnfj3s4tg5231cbgvtmrnc0000gn/T/ipykernel_28903/2604697549.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


### Function Decorator Approach

The `@tool` decorator in LangChain is used to easily register functions as tools that agents can use. It simplifies integration by marking the function for use in LangChain workflows without needing extra configuration.

In this case, the `@tool` decorator:
- Registers the `calculator` function as a callable tool for agents.
- Allows the agent to evaluate mathematical expressions safely by using `eval()` with restricted input handling.

This approach makes it easy to add modular, reusable tools to LangChain agents.


In [3]:
from langchain.tools import tool

@tool
def calculator(input_expr: str) -> int:
    """A simple calculator tool that evaluates a mathemtical expression and returns the result as an integer. Do not pass untrusted input."""
    return eval(input_expr.replace("^", "**"), {}, {})

tools = [calculator]

In [4]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

zero_shot_agent("what is (4.5*2.1)^2.2?")

/var/folders/61/42wnfj3s4tg5231cbgvtmrnc0000gn/T/ipykernel_28903/823011281.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  zero_shot_agent = initialize_agent(
/var/folders/61/42wnfj3s4tg5231cbgvtmrnc0000gn/T/ipykernel_28903/823011281.py:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  zero_shot_agent("what is (4.5*2.1)^2.2?")




> Entering new AgentExecutor chain...
I should use the calculator tool to evaluate this mathematical expression.
Action: calculator
Action Input: (4.5*2.1)^2.2
Observation: 139.94261298333066
Thought:I now know the final answer
Final Answer: 139.94

> Finished chain.


{'input': 'what is (4.5*2.1)^2.2?', 'output': '139.94'}

### Defining Tools for the Agent

In this section, we define two tools that the agent will use:

1. **QA Chat Tool**: A simple question-answering tool that takes a user message and generates a response using a large language model (LLM).
2. **RAG Chat Tool**: A more complex tool that handles document retrieval using Retrieval-Augmented Generation (RAG). This tool is specifically designed to retrieve and generate responses from environmental data.

In [10]:
# Define the QA Chat Tool
# This tool handles normal question-answering interactions by invoking the LLM
@tool
def qa_chat_tool(message: str) -> str:
    """Tool for handling normal question-answer (QA) interactions."""

    template = """Respond to the user message:
        message: {message}
        """
    prompt = PromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()
    response = chain.invoke(message)
    return response['output']  # Return the generated response

# Define the RAG Chat Tool
# This tool handles document retrieval and generation using RAG from environmental data
@tool
def rag_chat_tool(message: str) -> str:
    """Tool for handling document retrieval and generating responses using RAG from environmental data."""
    
    # Initialize the Retrieval-Augmented Generation (RAG) engine
    rag_engine = RagEngine()
    
    # Use the RAG engine to return an answer to the user's query
    response = rag_engine.return_answer(message)
    return response['output']


### Function Calling Application with Tools in LangChain

In this example, we are creating a function-calling agent using LangChain. The agent is set up with tools such as `qa_chat_tool` and `rag_chat_tool`, which can be used for question-answering and retrieval-augmented generation tasks. The agent is initialized as a zero-shot agent, meaning it can take actions based on descriptions of the tools provided to it without requiring training on specific tasks.

The `max_iterations` parameter limits the number of decision-making steps the agent can take before completing a task. Here's how to set up and initialize the agent with tools.


In [11]:
# Define the tools the agent will use
tools = [qa_chat_tool, rag_chat_tool]

# Initialize the zero-shot agent
# The agent is based on the "zero-shot-react-description" model, meaning it uses 
# reasoning and action descriptions to perform tasks without explicit training
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",  # Agent type for reasoning and decision making
    tools=tools,  # List of tools the agent will have access to
    llm=llm,  # The large language model (LLM) that powers the agent
    verbose=True,  # Set to True to see detailed output of the agent's decision-making process
    max_iterations=3  # Limit the agent to 3 iterations when performing actions
)

# You can now use zero_shot_agent to run function calls based on input queries

In [12]:
#This one may take a while oops
zero_shot_agent("what is the cause of global warming?")



> Entering new AgentExecutor chain...
I should use the RAG chat tool to retrieve information on the cause of global warming.
Action: rag_chat_tool
Action Input: "What is the cause of global warming?"

In [9]:
zero_shot_agent("What type of food is an apple?")



> Entering new AgentExecutor chain...
I should use the QA tool to get a straightforward answer to this question.
Action: qa_chat_tool
Action Input: "What type of food is an apple?"
Observation: An apple is a type of fruit. It is a sweet and crunchy fruit that is commonly eaten as a snack or used in various recipes.
Thought:I have the answer to the question about the type of food an apple is.
Final Answer: An apple is a type of fruit.

> Finished chain.


{'input': 'What type of food is an apple?',
 'output': 'An apple is a type of fruit.'}

### 2. Your Turn

Now, create a function-calling application of your own. You are encouraged to use external APIs to create robust tools.

In [ ]:

@tool
def MyTool(___):
    "A description of the tool"
